In [134]:
import re
from bs4 import BeautifulSoup, NavigableString
from notion_client import Client

NOTION_ACCESS_TOKEN = "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"
notion = Client(auth=NOTION_ACCESS_TOKEN)

response = notion.users.list()
print(response)

top_id = "177e700eb51580bfaf7bfaddd97cbc78"

{'object': 'list', 'results': [{'object': 'user', 'id': 'c3444096-12f7-4776-82ad-15bc3367c98e', 'name': 'Tat', 'avatar_url': None, 'type': 'person', 'person': {'email': 'taikunn12740pqp@icloud.com'}}, {'object': 'user', 'id': 'f5f588bd-b685-4634-925b-30173ad3d40c', 'name': 'TestIntegration', 'avatar_url': None, 'type': 'bot', 'bot': {'owner': {'type': 'workspace', 'workspace': True}, 'workspace_name': 'TatさんのNotion'}}, {'object': 'user', 'id': '178d872b-594c-810f-a5b6-00271dfa1976', 'name': 'ChatGPT to Notion', 'avatar_url': 'https://s3-us-west-2.amazonaws.com/public.notion-static.com/0744ff65-df85-4479-a95d-580f3b1f0684/icon.png', 'type': 'bot', 'bot': {}}], 'next_cursor': None, 'has_more': False, 'type': 'user', 'user': {}, 'request_id': '3aa2f9cc-79c4-4f1c-bad1-e7c96363abab'}


In [135]:

response = notion.blocks.children.list(block_id=top_id)

for block in response.get("results", []):
    print(block)

{'object': 'block', 'id': '177e700e-b515-8045-bf48-cc725679365e', 'parent': {'type': 'page_id', 'page_id': '177e700e-b515-80bf-af7b-faddd97cbc78'}, 'created_time': '2025-01-10T21:07:00.000Z', 'last_edited_time': '2025-01-13T10:50:00.000Z', 'created_by': {'object': 'user', 'id': 'c3444096-12f7-4776-82ad-15bc3367c98e'}, 'last_edited_by': {'object': 'user', 'id': 'f5f588bd-b685-4634-925b-30173ad3d40c'}, 'has_children': True, 'archived': False, 'in_trash': False, 'type': 'child_page', 'child_page': {'title': 'API_test'}}
{'object': 'block', 'id': '177e700e-b515-8078-b845-f58bc3c39a1c', 'parent': {'type': 'page_id', 'page_id': '177e700e-b515-80bf-af7b-faddd97cbc78'}, 'created_time': '2025-01-10T21:09:00.000Z', 'last_edited_time': '2025-01-10T21:44:00.000Z', 'created_by': {'object': 'user', 'id': 'c3444096-12f7-4776-82ad-15bc3367c98e'}, 'last_edited_by': {'object': 'user', 'id': 'c3444096-12f7-4776-82ad-15bc3367c98e'}, 'has_children': True, 'archived': False, 'in_trash': False, 'type': 'chil

In [137]:
def extract_fragment(text):
    # 正規表現で最初の <!--StartFragment--> と最後の <!--EndFragment--> を見つける
    start_pattern = r'<!--StartFragment-->'
    end_pattern = r'<!--EndFragment-->'

    # 文字列全体にわたって両方のタグの位置を探す
    start_match = re.search(start_pattern, text)
    end_match = re.search(end_pattern, text)

    if start_match and end_match:
        # 開始タグの終了位置と終了タグの開始位置を取得
        start_pos = start_match.end()
        end_pos = text.rfind(end_pattern)
        
        # 外側のペアに囲まれた文字列を取得
        return text[start_pos:end_pos].strip()
    else:
        return text

In [138]:
clip = Clip()
# print(clip.get_formats())
# clip.disp()
html = clip[49405].decode()
html = extract_fragment(html)


soup = BeautifulSoup(html, "lxml", preserve_whitespace_tags={"textarea", "p", "h1", "h2", "h3", "code", "strong"})
print(soup.prettify())



<html>
 <body>
  <p>これがインラインで使われる場合：
「直角三角形の辺の長さは、<span class="katex"><span class="katex-mathml"><math xmlns="http://www.w3.org/1998/Math/MathML"><semantics><mrow><msup><mi>a</mi><mn>2</mn></msup><mo>+</mo><msup><mi>b</mi><mn>2</mn></msup><mo>=</mo><msup><mi>c</mi><mn>2</mn></msup></mrow><annotation encoding="application/x-tex">a^2 + b^2 = c^2</annotation></semantics></math></span><span aria-hidden="true" class="katex-html"><span class="base"><span class="strut"></span><span class="mord"><span class="mord mathnormal">a</span><span class="msupsub"><span class="vlist-t"><span class="vlist-r"><span class="vlist"><span><span class="pstrut"></span><span class="sizing reset-size6 size3 mtight"><span class="mord mtight">2</span></span></span></span></span></span></span></span><span class="mspace"></span><span class="mbin">+</span><span class="mspace"></span></span><span class="base"><span class="strut"></span><span class="mord"><span class="mord mathnormal">b</span><span class="msupsub"><span

In [139]:
page_id = "177e700e-b515-8045-bf48-cc725679365e"

In [140]:
def append_object(parent_id, object):
    try:
        response = notion.blocks.children.append(parent_id, children=[object])
        if "results" in response and response["results"]:
            block_id = response["results"][0]["id"]
            print(f"Block created with ID: {block_id}")
            return block_id
        else:
            print(f"Failed to append block: {response}")
            return None
    except Exception as e:
        print(f"Error appending block: {e}")
        return None
        
def make_block(element, block_type, type="text", type_sub="content"):
    rich_text = process_inline_elements(element, type, type_sub)
    block_content = {
        # "object": "block",
        "type": block_type,
        block_type: {"rich_text": rich_text},
    }
    
    return block_content

def process_inline_elements(element, type, type_sub, annotations=None):
    # リッチテキストデータを格納するリスト
    rich_text = []

    # elementが文字列の場合の処理
    def text_elem(content, annotations, type, type_sub):
            return(
                {
                    "type": type,
                    type: {type_sub: content, "link": None},
                    "annotations": annotations.copy(),
                    "plain_text": content,
                    "href": None,
                }
            )

    if annotations is None:
        annotations = {
            "bold": False,
            "italic": False,
            "strikethrough": False,
            "underline": False,
            "code": False,
            "color": "default",
        }

    # タグの処理
    if element.name in ["strong", "b"]:
        annotations["bold"] = True
    if element.name in ["em", "i"]:
        annotations["italic"] = True
    if element.name in ["u", "ins"]:
        annotations["underline"] = True
    if element.name in ["s", "strike"]:
        annotations["strikethrough"] = True
    if element.name == "code":
        annotations["code"] = True
        annotations["color"] = "red"

    if element.name == "math" and not element.has_attr("display"):
        rich_text.append(text_elem(content, annotations, type="equation", type_sub="expression"))
        

    if isinstance(element, (str, NavigableString)):
        # 文字列またはNavigableStringの場合
        content = str(element).replace('\n', ' ').strip()
        if content:
            rich_text.append(text_elem(content, annotations, type, type_sub))
    else:
        # 子要素があるか確認
        has_children = any(not isinstance(child, (str, NavigableString)) for child in element.children)

        if not has_children and element.string:
            # 子要素がなく、テキストノードの場合
            content = element.string.replace('\n', ' ').strip()
            if content:
                rich_text.append(text_elem(content, annotations, type, type_sub))
        else:
            # 子要素の再帰処理
            for child in element.children:
                if child.name == "p":
                    return rich_text
                elif child.name not in ["h1", "h2", "h3", "ul", "ol", "li", "div", "pre", "math"]:
                    rich_text.extend(process_inline_elements(child, type, type_sub, annotations.copy()))

    return rich_text

def extract_code_block(block):
    language_div = block.find("div", class_="flex items-center text-token-text-secondary px-4 py-2 text-xs font-sans justify-between rounded-t-md h-9 bg-token-sidebar-surface-primary dark:bg-token-main-surface-secondary select-none")
    language = language_div.get_text(strip=True)
    if not language in [ "abap", "agda", "arduino", "ascii art", "assembly", "bash", "basic", "bnf", "c", "c#", "c++", "clojure", "coffeescript", "coq", "css", "dart", "dhall", "diff", "docker", "ebnf", "elixir", "elm", "erlang", "f#", "flow", "fortran", "gherkin", "glsl", "go", "graphql", "groovy", "haskell", "hcl", "html", "idris", "java", "javascript", "json", "julia", "kotlin", "latex", "less", "lisp", "livescript", "llvm ir", "lua", "makefile", "markdown", "markup", "matlab", "mathematica", "mermaid", "nix", "notion formula", "objective-c", "ocaml", "pascal", "perl", "php", "plain text", "powershell", "prolog", "protobuf", "purescript", "python", "r", "racket", "reason", "ruby", "rust", "sass", "scala", "scheme", "scss", "shell", "smalltalk", "solidity", "sql", "swift", "toml", "typescript", "vb.net", "verilog", "vhdl", "visual basic", "webassembly", "xml", "yaml", "java/c/c++/c#", "notionscript" ]:
        language = "plain text"

    code_element = block.find("code", class_=re.compile("^!whitespace-pre hljs language-"))
    code = code_element.text.strip()

    return language, code

def extract_math_block(block):
    math_block = block.find('annotation', {'encoding': 'application/x-tex'})
    expression = math_block.text

    return expression

def html_to_notion(parent_id, block):

    if isinstance(block, str):
        return
    block_id = None
    
    if block.name == "p":
        block_type = "paragraph"
        block_content = make_block(block, block_type)
        block_id = append_object(parent_id, block_content)

    elif block.name in ["h1", "h2", "h3", "h4", "h5", "h6"]:
        block_type = f"heading_{min(int(block.name[1]), 3)}"
        block_content = make_block(block, block_type)
        block_id = append_object(parent_id, block_content)

    elif block.name == "li":
        block_type = None
        parent_tag = block.parent.name if block.parent else None
        if block.find(True):  # 直下にタグがある場合
            first_child = block.contents[0]  # 直下の最初のタグを取得
            if first_child.name == "p":
                first_child.unwrap() # <p>タグのタグのみを削除
        if parent_tag == "ol":
            block_type = "numbered_list_item"
        else:
            block_type = "bulleted_list_item"
        block_content = make_block(block, block_type)
        block_id = append_object(parent_id, block_content)

    elif block.name == "pre":
        # try:
            code_language, code = extract_code_block(block)
            block_content = {
              "type": "code",
              "code": {
                "caption": [],
                    "rich_text": [{
                    "type": "text",
                    "text": {
                    "content": code
                    }
                }],
                "language": code_language
              }
            }
            block_id = append_object(parent_id, block_content)
        # except AttributeError:
            # return

    elif block.name == "math" and block.has_attr('display'):
        expression = extract_math_block(block)
        block_content = {
          "type": "equation",
          "equation": {
            "expression": expression,
            }
        }
        block_id = append_object(parent_id, block_content)

    elif block.name in ["html", "ul", "ol"]:
        block_id = parent_id
        
    elif block.name in ["body"]:
        block_id = parent_id
        # body直下の内容がブロック要素を含まないとき、出力できるように
        # 要改善 もう一周回す？
        if not any(((child.name in ["h1", "h2", "h3", "ul", "ol", "li", "div", "pre", "math"]), child.has_attr("display")) for child in block.children if child.name):
            block.name = "p"
            html_to_notion(block_id, block)
            return
            
    else:
        block_id = parent_id
            
        
    for child in block.children:
        html_to_notion(block_id, child)


In [ ]:
html_ele = BeautifulSoup(html, "lxml").body

print(html_ele)
html_to_notion(page_id, html_ele)

# print(list(html.children))

# print(html)

<body><p>これがインラインで使われる場合：
「直角三角形の辺の長さは、<span class="katex"><span class="katex-mathml"><math xmlns="http://www.w3.org/1998/Math/MathML"><semantics><mrow><msup><mi>a</mi><mn>2</mn></msup><mo>+</mo><msup><mi>b</mi><mn>2</mn></msup><mo>=</mo><msup><mi>c</mi><mn>2</mn></msup></mrow><annotation encoding="application/x-tex">a^2 + b^2 = c^2</annotation></semantics></math></span><span aria-hidden="true" class="katex-html"><span class="base"><span class="strut"></span><span class="mord"><span class="mord mathnormal">a</span><span class="msupsub"><span class="vlist-t"><span class="vlist-r"><span class="vlist"><span><span class="pstrut"></span><span class="sizing reset-size6 size3 mtight"><span class="mord mtight">2</span></span></span></span></span></span></span></span><span class="mspace"></span><span class="mbin">+</span><span class="mspace"></span></span><span class="base"><span class="strut"></span><span class="mord"><span class="mord mathnormal">b</span><span class="msupsub"><span class="vli